<a href="https://colab.research.google.com/github/RSNA/AI-Deep-Learning-Lab-2023/blob/main/sessions/nlp-text-classification/RSNA23_ACR_contrast_manual_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSNA 2023: Deep Learning Lab
## Chat with the ACR Contrast Manual
### Using LLMs and retrieval-augmented generation (RAG)

`Retrieval-augmented generation (RAG)` is a method of querying existing data to improve the responses of large language models (LLMs) to questions requiring factual and/or specialized knowledge.

RAG is a two-step process where relevant information is first retrieved from a special database, called a `vector database`, that stores `chunks` of text along with the embeddings of that text, typically from a Transformer or LLM.

Encoding a question or prompt from a user with the same model used to generate the embeddings for the vector DB, one can use `similarity search` to find relevant chunks of text that can then be presented to an LLM as part of the prompt including the original question.

In this way, you can "chat" with any document or database you like. One popular application of this is `manual-as-a-service`.

## Module Overview
In this module, we will:
1. Split the [ACR Contrast Manual](https://www.acr.org/-/media/ACR/Files/Clinical-Resources/Contrast_Media.pdf) into chunks.
2. Create embeddings for the chunks with the MedCPT Article Encoder.
3. Store the `chunk:embedding` pairs in a vector DB.
4. Set up a RAG Q&A pipeline with the `LlamaIndex` framework.
5. Test out our RAG Q&A pipeline to "chat" with the ACR Contrast Manual.

## References
* https://www.promptingguide.ai/techniques/rag
* MedCPT ArXiv paper: https://arxiv.org/abs/2307.00589
    - MedCPT Article Encoder on Hugging Face: https://huggingface.co/ncbi/MedCPT-Article-Encoder
* LlamaIndex LLM Application Framework: https://docs.llamaindex.ai/en/stable/index.html
    - LlamaIndex `llama-cpp-python` Integration: https://docs.llamaindex.ai/en/stable/examples/llm/llama_2_llama_cpp.html
* Llama2-7B-Chat on Hugging Face: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

In [ ]:
# @title Installing required libraries
# @markdown This cell will take approximately 4 minutes to run.<br><br>When the cell finishes running the `Runtime` will be restarted. This will appear as an error saying that your session crashed for an unknown reason.
# @markdown <br><br>Don't worry. This is expected.

%%capture
!pip uninstall numpy -y
!pip install numpy==1.25
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.11 --force-reinstall --upgrade --no-cache-dir
!pip install -U \
    llama-index==0.8.69.post2 \
    huggingface-hub==0.19.3 \
    transformers==4.35.2 \
    pypdf==3.17.1

import os
os.kill(os.getpid(), 9)

In [1]:
# @title Import the necessary libraries and functions
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.readers import PDFReader
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

from pathlib import Path

In [2]:
# @title Download the ACR Contrast Manual

!mkdir pdfs
!wget -qP /content/pdfs https://www.acr.org/-/media/ACR/Files/Clinical-Resources/Contrast_Media.pdf
# !wget -qP /content/pdfs https://www.acr.org/-/media/ACR/Files/Radiology-Safety/MR-Safety/Manual-on-MR-Safety.pdf

In [3]:
# @title Load the PDF

pdf_folder_path = Path("/content/pdfs")
documents = PDFReader().load_data(pdf_folder_path/"Contrast_Media.pdf")
# documents = [PDFReader().load_data(pdf_folder_path/fn) for fn in list(pdf_folder_path.glob('**/*.pdf'))]

In [4]:
# @title Obtain our embedding model from Hugging Face Hub

embed_model = HuggingFaceEmbedding(model_name="ncbi/MedCPT-Article-Encoder")

(…)Article-Encoder/resolve/main/config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

(…)coder/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

(…)T-Article-Encoder/resolve/main/vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

(…)icle-Encoder/resolve/main/tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

(…)e-Encoder/resolve/main/added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

(…)der/resolve/main/special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
# @title We'll be using the Llama-2-7B-Chat model via the Llama.cpp integration

model_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=512,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=2048,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 36},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 4081.0


3892it [00:25, 154.41it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
# @title Create our `ServiceContext` to specify our custom embeddings and LLM
# @markdown You can experiment with the `chunk_size` parameter to determine it's effect on inference speed and effective retrieval.

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    chunk_size=1000
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# @title Create our `VectorStoreIndex` Query Engine from the PDF and our `ServiceContext`

index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

# @markdown Another hyperparameter to experiment with is `similarity_top_k`. This is the number of chunks of text that will be retrieved from the `VectorStoreIndex` for each query.
query_engine = index.as_query_engine(similarity_top_k=3)

In [8]:
# @title Test our RAG Q&A pipeline

response = query_engine.query("What is the GFR threshold below which IV contrast should be withheld in a patient with acute kidney injury?");
print(response)

  Based on the provided context information, there is no agreed-upon threshold of eGFR elevation or declination beyond which the risk of CI-AKI is considered so great that intravascular iodinated contrast medium should never be administered. In fact, each contrast medium administration always implies a risk-benefit analysis for the patient, and contrast medium administration for all patients should always be taken in the clinical context, considering all risks, benefits, and alternatives [2, 6].
Some practices have advocated stratification of potential risk by eGFR instead of serum creatinine because it is a better indicator of baseline renal function [49, 50]. However, there is limited data on this topic, and no agreed-upon threshold has been established. Therefore, the decision to withhold IV contrast medium in a patient with acute kidney injury should be made on an individual basis, taking into account the patient's specific clinical context and renal function.
In summary, there is 

Play around with different queries and see how well the model responds.

You could even try different embedding models and LLMs available on Hugging Face.

In [9]:
response = query_engine.query("Is gadoxetate a group II gadolinium based contrast material?");
print(response)

Llama.generate: prefix-match hit


  Based on the provided context and without any prior knowledge, I can confirm that gadoxetate is indeed a group II gadolinium-based contrast material. According to the information provided in the context, gadoxetate is listed as a group II gadolinium-based contrast agent, specifically Gd-EOB-DTPA.


In [10]:
response = query_engine.query("What group does gadoxetic acid belong to?");
print(response)

Llama.generate: prefix-match hit


  Based on the provided context information, gadoxetic acid belongs to the group of macrocyclic non-ionic GBCAs.


In [11]:
response = query_engine.query("What considerations should I be aware of when giving IV contrast to a patient taking metformin?");
print(response)

Llama.generate: prefix-match hit


  As an honest and respectful assistant, I must inform you that there is no direct evidence or consensus in the medical literature to suggest that metformin increases the risk of contrast-induced acute kidney injury (CI-AKI) when given intravenously. However, there are some studies that have suggested a possible association between metformin use and an increased risk of lactic acidosis in patients who develop AKI while taking the drug. Therefore, it is important to monitor the patient's renal function closely and be aware of the signs and symptoms of lactic acidosis if they are receiving IV contrast media.
In terms of considerations for giving IV contrast to a patient taking metformin, here are some general guidelines:
1. Assess the patient's baseline renal function: Before administering IV contrast, it is important to assess the patient's baseline renal function to determine their risk of developing AKI. This can be done by measuring the patient's serum creatinine level before adminis

## Inspecting the Retrieved Source Texts

Sometimes, it can be helpful to see what source texts were provided as context for the LLM to answer your query. This can help in troubleshooting and determining whether to increase the number of texts retrieved for each query.

In [12]:
response = query_engine.query("Should metforming be discontinued when giving IV gadolinium-based contrast?");
print(response)

Llama.generate: prefix-match hit


  Based on the provided context and without any prior knowledge, I would advise that metformin should not be discontined when giving IV gadolinium-based contrast. There is no evidence to suggest that metformin increases the risk of contrast-induced acute kidney injury (CI-AKI) in patients with chronic kidney disease (CKD). In fact, a study published in the Journal of the American Society of Nephrology found that metformin was associated with a lower risk of AKI compared to other diabetes medications.
However, it is important to note that CI-AKI can occur in patients with CKD who receive IV gadolinium-based contrast, regardless of the presence of metformin. The risk of CI-AKI is generally low, but it can be higher in patients with more severe kidney impairment or those who are dehydrated or have preexisting kidney disease.
Therefore, I would recommend that patients with CKD who require IV gadolinium-based contrast should be assessed for their risk of CI-AKI and monitored closely for sig

In [13]:
sources = response.get_formatted_sources(length=1000)
print(sources)

> Source (Doc id: a594aa79-dc12-42df-980e-0682a4ba2dd3): Preface  
2 
 
 
  
 
  
 
 
 
ACR Manual 
On 
Contrast  Media  
 
 
2023  
 
 
 
ACR Committee  
on   
                Drugs  and Contrast  Media

> Source (Doc id: a42f803e-bbb6-49c0-a6d5-ec3008cf456f): CONTRAST -ASSOCIATED ACUTE KIDNEY INJURY AND CONTRAST -INDUCED ACUTE KIDNEY INJURY IN ADULTS   44   
• Personal history of renal disease, including:  
Known chronic kidney disease (CKD)  
   Remote history of AKI  
Dialysis  
Kidney surgery 
Kidney ablation  
Albuminuria  
•  History of diabetes mellitus (optional)  
• Metformin or metformin -containi ng drug combinations1 
Patients who are scheduled for a routine intravascular study but do not have one of the above risk factors do not 
require a baseline serum creatinine determination before iodinated contrast medium administration.  
 
Morbidity and Mortality  
 The clinical course of CA -AKI (and, presumably, CI-AKI) depends on baseline renal function, coexisting risk 
factor

## More Test Queries

In [14]:
response = query_engine.query("What are some of the concerns regarding children and IV contrast?");
print(response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Thank you for providing additional context. Based on the new information provided, here is a refined answer to your original query:
The concerns regarding children and IV contrast are as follows:
1. Extravasation-related signs or symptoms: Children are more susceptible to extravasation-related complications due to their smaller body size and immature circulatory system. It is essential to monitor children closely during and after contrast media injection to detect any signs of extravasation, such as severe pain, progressive swelling or pain, altered tissue perfusion, and skin ulceration or blistering.
2. Large volume extravasation: Children may be more likely to experience large volume extravasation due to their smaller body size, which can lead to serious complications such as compartment syndrome. It is crucial to use the smallest possible dose of contrast media necessary for the imaging study and to closely monitor children during the injection process.
3. Discharging patients: Ch

In [15]:
response = query_engine.query("When should surgical consultation be obtained after contrast extravasation?");
print(response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Thank you for providing additional context. Based on the updated information, the answer to the query "When should surgical consultation be obtained after contrast extravasation?" is:
Surgical consultation should be obtained immediately if any of the following signs or symptoms develop after a contrast extravasation:
* Severe pain that worsens over time
* Progressive swelling or pain that persists for more than 24 hours
* Altered tissue perfusion as evidenced by decreased capillary refill that does not improve with elevation of the affected limb
* Change in sensation in the affected limb that is not resolving
* Worsening passive or active range of motion that is not improving
* Skin ulceration or blistering that is not healing

It is important to note that an extravasation volume threshold (such as estimated volumes exceeding 100 or 150 mL) should not be used to indicate the need for surgical consultation. Instead, signs and symptoms should be evaluated and clinical follow-up should 

In [16]:
response = query_engine.query("Can I power-inject contrast into a PICC line?");
print(response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Thank you for providing additional context. Based on the new information provided, it is not recommended to power-inject contrast into a PICC line as it can increase the risk of extravasation injury. The manufacturer recommendations should be followed, and the flow limit should be noted before using a central venous line for power injection. It is important to ensure that the port site is certified as power injectable, and meticulous intravenous line insertion technique, angiocatheters, confirming position by aspirating blood, flushing an inserted catheter with a test injection of saline, and carefully securing the inserted catheter should be used to reduce the risk of extravasation injuries.
Original Answer: Based on the provided context information, it is not recommended to power-inject contrast into a PICC line. The risk of extravasation injury is higher when using a PICC line for power injection compared to other central venous catheters. In addition, manufacturer recommendations

In [17]:
response = query_engine.query("When should kidney function be checked prior to giving iodinated contrast?");
print(response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Thank you for providing additional context. Based on the updated information, it is important to check kidney function before administering iodinated contrast to patients who are at risk of developing contrast-induced nephropathy (CIN). The recommended tests for renal function assessment include serum creatinine and estimated glomerular filtration rate (eGFR). If the results indicate abnormal kidney function, the administration of iodinated contrast should be delayed or alternative imaging modalities should be considered.
In particular, patients with pre-existing kidney disease, diabetes, or heart failure are at higher risk of developing CIN and should have their kidney function assessed before receiving iodinated contrast. Additionally, patients taking medications that can affect kidney function, such as non -steroidal anti -inflammatory drugs (NSAIDs) or angiotensin -converting enzyme (ACE) inhibitors, may also be at higher risk of CIN and should have their kidney function assessed

In [18]:
response = query_engine.query("In what situations should kidney function be checked prior to iodinated contrast?");
print(response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Thank you for providing additional context. Based on the updated information, it is important to check kidney function prior to administering iodinated contrast in the following situations:
1. Patients with end-stage renal disease (ESRD) on chronic dialysis: In these patients, checking kidney function before administering iodinated contrast is crucial to avoid any potential complications.
2. Patients at risk of nephrogenic systemic fibrosis (NSF): Group I agents (iodinated contrast media) are contraindicated in patients with a history of NSF, and checking kidney function before administering these agents is essential to minimize the risk of developing NSF.
3. Patients with acute kidney injury (AKI): In patients with AKI, administering iodinated contrast can further compromise kidney function, and checking kidney function beforehand is crucial to identify any potential issues.
4. Patients taking metformin: Metformin use has been associated with an increased risk of developing NSF, and